<a href="https://colab.research.google.com/github/shreyamanapure/comment-analysis-using-BERT/blob/main/Comments_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. GREAT!!',return_tensors='pt')

In [6]:
tokens[0]

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   106,   106,   102])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great!! [SEP]'

In [8]:
result = model(tokens)

In [9]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.8237, -1.2152,  1.4776,  1.9598,  0.4585]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
result.logits

tensor([[-2.8237, -1.2152,  1.4776,  1.9598,  0.4585]],
       grad_fn=<AddmmBackward0>)

In [11]:
int(torch.argmax(result.logits))+1

4

In [13]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment*.')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [23]:
reviews[0]

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [24]:
import numpy as np
import pandas as pd

In [25]:
df = pd.DataFrame(np.array(reviews),columns=['reviews'])

In [26]:
df.tail()

,reviews
5,Out of all the restaurants that I tried in Syd...
6,We came here on a Thursday night @ 5pm and by ...
7,The food is fresh and tasty. The scallop cevi...
8,Have been here twice and have absolutely loved...
9,I was pleasantly surprised at what a great job...


In [28]:
df['reviews'].iloc[0]

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [29]:
def sentiment_score(reviews):
  tokens = tokenizer.encode(reviews,return_tensors='pt')
  results = model(tokens)
  return int(torch.argmax(result.logits))+1

In [30]:
sentiment_score(df['reviews'].iloc[1])

4

In [32]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [34]:
df

,reviews,sentiment
0,Hotter than I imagine myself to be and firing ...,4
1,The food and service here was really good. It...,4
2,Visiting from Texas and decided to give this r...,4
3,"Great atmosphere, attentive service, solid mar...",4
4,Don't come here expecting legit Mexican food b...,4
5,Out of all the restaurants that I tried in Syd...,4
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,4
9,I was pleasantly surprised at what a great job...,4


In [36]:
 df['reviews'].iloc[4]

"Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someone down to pay the bill. The watermelon salad was tasty but not complex, tossed with a few cucumbers and pistachios. The corn lollipops with spicy mayo sauce were probably the best dish. The beef empanadas were cold and average though the salsa was an interesting pickled onion. Definitely skip the octopus dish- it was dry, over cooked and lacked flavor though the presentation looked promising. It came with squares of polenta and dallops of potato mash with a dramatic squid ink streak across the dish bu